<a href="https://colab.research.google.com/github/dhamu2908/Review-Classifier/blob/main/Review_NaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [395]:
import pandas as pd
import numpy as np
import csv
import re
import torch
import math
from collections import defaultdict
import seaborn as sns

In [396]:
pd.options.mode.chained_assignment=None

In [397]:
df=pd.read_csv("zomato_reviews.csv")

In [398]:
print(df.head())

   Unnamed: 0  rating                                             review
0           0       5                                               nice
1           1       5  best biryani , so supportive staff of outlet ,...
2           2       4     delivery boy was very decent and supportive.👌👍
3           3       1  worst biryani i have tasted in my life, half o...
4           4       5  all food is good and tasty . will order again ...


In [399]:
df1=df[['rating','review']]

In [400]:
print(df1.head())

   rating                                             review
0       5                                               nice
1       5  best biryani , so supportive staff of outlet ,...
2       4     delivery boy was very decent and supportive.👌👍
3       1  worst biryani i have tasted in my life, half o...
4       5  all food is good and tasty . will order again ...


In [401]:
df1.replace({1:'bad',2:'okay',3:'okay',4:'good',5:'good'},inplace=True)

In [402]:
print(df1.head())

  rating                                             review
0   good                                               nice
1   good  best biryani , so supportive staff of outlet ,...
2   good     delivery boy was very decent and supportive.👌👍
3    bad  worst biryani i have tasted in my life, half o...
4   good  all food is good and tasty . will order again ...


In [403]:
df1['rating'].value_counts()

,count
rating,
good,2746
bad,1891
okay,842


In [404]:
df1.isnull().sum()

,0
rating,0
review,1


In [405]:
df1['review'].fillna(2,inplace=True)
df1['rating'].str.replace(r'[^a-zA-Z\s]','',regex=True)

<ipython-input-405-ecda378d84e1>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df1['review'].fillna(2,inplace=True)


,rating
0,good
1,good
2,good
3,bad
4,good
...,...
5474,good
5475,good
5476,good
5477,bad


In [406]:
print(df1.head)

<bound method NDFrame.head of      rating                                             review
0      good                                               nice
1      good  best biryani , so supportive staff of outlet ,...
2      good     delivery boy was very decent and supportive.👌👍
3       bad  worst biryani i have tasted in my life, half o...
4      good  all food is good and tasty . will order again ...
...     ...                                                ...
5474   good                                           complain
5475   good  it took 1 hour to assign valvet and thn prepar...
5476   good  took for an hour to prepare 3 khawsa, which in...
5477    bad  very very late, littrally did time pass and it...
5478    bad  Taste was stale and they give only 5 pieces in...

[5479 rows x 2 columns]>


In [407]:
df1.isnull().sum()

,0
rating,0
review,0


In [408]:
train_size = int(0.8 * len(df1))
val_size = int(0.9 * len(df1))

X_train = df1.iloc[:train_size]["review"]
y_train = df1.iloc[:train_size]["rating"]

X_val = df1.iloc[train_size:val_size]["review"]
y_val = df1.iloc[train_size:val_size]["rating"]

X_test = df1.iloc[val_size:]["review"]
y_test = df1.iloc[val_size:]["rating"]

In [409]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)
print(X_train)

(4383,)
(4383,)
(548,)
(548,)
(548,)
(548,)
0                                                    nice
1       best biryani , so supportive staff of outlet ,...
2          delivery boy was very decent and supportive.👌👍
3       worst biryani i have tasted in my life, half o...
4       all food is good and tasty . will order again ...
                              ...                        
4378                               Not recommended to eat
4379                  fantastic and testy good for me to.
4380            quantity is too too less ,refund required
4381                   it's not shorba. it's mutton curry
4382    the restaurant should have tandoori roti.. as ...
Name: review, Length: 4383, dtype: object


In [410]:
dict=defaultdict(int)
for i in X_train:
  #print(i)
  try:
    words=i.split()
    for word in words:
      dict[word]+=1
  except:
    pass

In [411]:
"""for i in dict:
  print(i,end=" : ")
  print(dict[i])"""
print(len(dict))

7425


In [412]:
good_prob=defaultdict(int)
bad_prob=defaultdict(int)
okay_prob=defaultdict(int)
good=0
bad=0
okay=0
alpha=1

In [413]:
for i in range(len(X_train)):
  #print(X_train[i])
  try:
    words=X_train[i].split()
    for word in words:
      if word not in good_prob:
        good_prob[word]+=alpha
      if word not in bad_prob:
        good_prob[word]+=alpha
      if word not in okay_prob:
        good_prob[word]+=alpha

      if y_train[i]=='good':
        good_prob[word]+=1
        pos+=1
      elif y_train[i]=='bad':
        bad_prob[word]+=1
        bad+=1
      else:
        okay_prob[word]+=1
        okay+=1
  except:
    pass

In [414]:
good_prob={key:val/(good+alpha) for key,val in dict.items()}
bad_prob={key:val/(bad+alpha) for key,val in dict.items()}
okay_prob={key:val/(okay+alpha) for key,val in dict.items()}

In [415]:
y_pred=[]
c=0
for i in X_train:
  try:
    words=i.split()
    c+=1
    g=b=o=1
    for word in words:
      g*=good_prob[word]
      b*=good_prob[word]
      o*=good_prob[word]
    if (max(g,b,o))==g:
      res="good"
    elif (max(g,b,o))==b:
      res="bad"
    else:
      res="okay"
    y_pred.append(res)
  except:
    y_pred.append("good")
count=0
print(len(y_pred))
print(len(y_train))
for i in range(len(y_train)):
  if y_train[i]==y_pred[i]:
    count+=1
Accuracy=count/len(y_train)
print(Accuracy*100)

4383
4383
46.634725074150126
